# TinyURL System Design

Course from [TinyURL](https://www.educative.io/courses/grokking-the-system-design-interview/m2ygV4E81AR), providing short aliases redirecting to long URLs.

# Purpose

URL shortening is used to create shorter aliases for long URLs.

1. Save space
2. Avoid mistype

# Requirements and Goals

Before designing the system, the requirements should be clarified clearly.

## Functional Requirements

1. Given a URL, system should generate a shorter and unique alias of it. This link should be short enough to be easily copied and pasted into applications.
2. When users access a short link, our service should redirect them to the original link.
3. Users should optionally be able to pick a custom short link for their URL.
4. Links will expire after a standard default timespan, which users should be able to specify.

## Non-Functional Requirements

1. Highly available
2. Real-time response
3. Shortened links should not be guessable

## Extended Requirements

1. Analytics
2. Open REST APIs

# Capacity Estinmation and Constraints

The system should be read-heavy. There should be lots of redirection requests compared to new URL shortenings.

At first we can assume **the ratio is 100:1** between read and write.

## Traffic Estimates

Assume we have 500M new shortenings per month, we can expect 50B redirections during the same period:

$$
100 * 500M => 50B
$$

And Queries Per Second (QPS) can be estimated:

$$
500M / (30 days * 24 hours * 3600 seconds) \approx 200 URLs/s
$$

Based on the 100:1 read/write ratio, URLs redirections per second will be:

$$
100 * 200 URLs/s = 20K/s
$$

## Storage Estimates

Assume the lifetime of URL shortening request is 5 years. We expect to have 500M new URLs every month, and thus the total requests will be 30B:

$$
500M * 5 years * 12 months = 30B
$$

If each stored object is 500 bytes, we will need 15TB for total storage:

$$
30B * 500 bytes = 15TB
$$

## Bandwidth Estimates

Based on the QPS 200 URLs per second, we can estimate the bandwidth is 100KB per second:

- Write:

$$
200 * 500 bytes = 100 KB/s
$$

- Read:

$$
20K * 500 bytes \approx 10MB/s
$$

## Memory Estimates

Because some of the URLs are more popular than most of them, and we can use Pareto Rule (80/20) first to cache the 20% URLs.

Assume we have 20K requests per second, we expect to get 1.7B requests per day:

$$
20K * 3600 * 24 \approx 1.7B
$$

To cache 20% of the 1.7B requests, we will require 170GB to cache hot URLs:

$$
1.7B * 20\% * 500 bytes \approx 170GB
$$

In the real condition, there should be many duplicate URLs, and thus our actual memory usage might be less than 170GB.

## High-Level Estimates

To sum up all the estimations above:

| Types of URLs | Time Estimates |
| --- | --- |
| New URLs | 200/s |
| URL redirections | 20K/s |
| Incoming Data | 100KB/s |
| Outcoming Data | 10MB/s |
| Storage for 5 years |  15TB |
| Memory for cache | 170GB |

# System APIs

We can have REST APIs to expose the functionality of our service, the first is to define create/delete.

`createURL(api_dev_key, original_url, custom_alias=None, user_name=None, expire_date=None)`

- Parameters:
    - api_dev_key: The API developer key of a registered account, so as to throttle users based on their allocated quota.
    
- Returns: (string)
Returns the shortened URL; otherwise, returns an error code.
    
`deleteURL(api_dev_key, url_key)`

## Detect & Prevent Abuse

A malicious user can put us out of business by consuming all URL keys. To prevent abuse, we can limit users via their api_dev_key.

# Database Design

There are some principals for us to follow:

1. We need to store billions of records.
2. Each object we store is small (less than 1KB).
3. There are no relationship between records.
4. Our service is read-heavy.

## Database Schema

We will need two tables:

1. URL

| PK | **Hash: varchar(16)** |
| --- | --- |
| | OriginalURL: varchar(512) |
| | CreationDate: datetime |
| | ExpirationDate: datetime |
| | UserID: int |

2. User

| PK | **UserID: int** |
| --- | --- |
| | Name: varchar(20) |
| | Email: varchar(32) |
| | CreationDate: datetime |
| | LastLogin: datetime |

### Datebase to Use

Since we don't need to store the relationships betweeb objects - a NoSQL like DynamoDB, Cassandra, Riak, or MongoDB is a better choice, and it is easier to scale than SQL.

# Basic System Design and Algorithm

Our goal is to generate a shortended URL.

## Encoding Actual URL

It is straightforward to encode original URL using a hash function (e.g., MD5 or SHA256). The encoding could be base36(\[a-z, 0-9\]) or base62(\[A-Z, a-z, 0-9\]) and if we add '+' and '/' we can use Base64 encoding. Afterwards, we have to determine the length of the short key.

Using base64 encoding, a 6 letters long key would result in $64^6 \approx 68.7B$ possible strings.
Using base64 encoding, a 8 letters long key would result in $64^8 \approx 281T$ possible strings.

With 68.7B unique strings, it is suffice for our system.

### Potential Issues

1. If users enter the same URL, they should not receive the same shortended URL.
2. What if parts of the URL are URL-encoded?

### Workaround

1. Appending a sequence number
    - Might cause overflow
    - Slows down the performance
2. Append user ID
    - User might be anynomous
    
## Generating Keys Offline

We can have a standalone **Key Generation Service (KGS)** to generate random six-letter strings beforehand and stores them in a database (key-DB).

### Potential Issues

1. Can concurrency cause problems?

### Workaround

1. Group keys into not-used and used two groups.
    - System uses only not-used keys, and move them to used group.
2. 